# Human Pose Classification using GCN

This notebook implements the training pipeline for classifying human poses using Graph Convolutional Networks (GCN).

In [ ]:
# Import required libraries
import torch
from torch_geometric.loader import DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import os

# Import our modules
import sys
sys.path.append('src')
from data_processing import load_dataset
from gcn_model import PoseGCN

print("All libraries imported successfully!")

## Check Data Paths

In [ ]:
# Define and verify data paths
data_dir = os.path.join(os.getcwd(), 'data_v2')
annotation_dir = os.path.join(os.getcwd(), 'annotations_v2')

# Check if directories exist
print(f"Checking data directory: {data_dir}")
print(f"Exists: {os.path.exists(data_dir)}")

print(f"\nChecking annotation directory: {annotation_dir}")
print(f"Exists: {os.path.exists(annotation_dir)}")

# Check annotation files
annotation_files = [
    't5-sherul-300-195-correct.json',
    'lumbar-K-1.1-160.json'
]

print("\nChecking annotation files:")
for file in annotation_files:
    file_path = os.path.join(annotation_dir, file)
    print(f"{file}: {'Exists' if os.path.exists(file_path) else 'Missing'}")

## Define Training and Evaluation Functions

In [28]:
def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for data in tqdm(train_loader, desc='Training', leave=False):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        loss = torch.nn.functional.nll_loss(output, data.y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

In [29]:
def evaluate(model, loader, device):
    model.eval()
    predictions = []
    labels = []
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            pred = output.max(dim=1)[1]
            
            predictions.extend(pred.cpu().numpy())
            labels.extend(data.y.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return accuracy, precision, recall, f1

In [30]:
def plot_metrics(train_losses, val_metrics):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot training loss
    ax1.plot(train_losses, 'b-', label='Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)
    ax1.legend()
    
    # Plot validation metrics
    epochs = range(len(val_metrics['accuracy']))
    ax2.plot(epochs, val_metrics['accuracy'], 'g-', label='Accuracy')
    ax2.plot(epochs, val_metrics['precision'], 'r-', label='Precision')
    ax2.plot(epochs, val_metrics['recall'], 'b-', label='Recall')
    ax2.plot(epochs, val_metrics['f1'], 'y-', label='F1-Score')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Score')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

## Setup Training

In [ ]:
try:
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load datasets using absolute paths
    train_dataset, val_dataset = load_dataset(
        root_dir=data_dir,
        annotation_dir=annotation_dir
    )

    print(f"Dataset loaded: {len(train_dataset)} training samples, {len(val_dataset)} validation samples")

    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    model = PoseGCN(num_node_features=2).to(device)  # 2 features: x, y coordinates
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("Model and optimizer initialized!")
    
except Exception as e:
    print(f"Error during setup: {str(e)}")
    raise

## Training Loop

In [ ]:
# Training parameters
num_epochs = 100
best_val_acc = 0
train_losses = []
val_metrics = {
    'accuracy': [], 'precision': [], 'recall': [], 'f1': []
}

# Training loop
try:
    for epoch in range(num_epochs):
        # Train
        train_loss = train(model, train_loader, optimizer, device)
        train_losses.append(train_loss)
        
        # Evaluate
        val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, device)
        
        # Store metrics
        val_metrics['accuracy'].append(val_acc)
        val_metrics['precision'].append(val_prec)
        val_metrics['recall'].append(val_rec)
        val_metrics['f1'].append(val_f1)
        
        # Print progress
        print(f'Epoch {epoch+1:03d}:')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Val Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, '
              f'Recall: {val_rec:.4f}, F1: {val_f1:.4f}')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'val_metrics': val_metrics,
                'train_losses': train_losses
            }, 'models/model-best_model.pth')
            print(f"Saved best model with validation accuracy: {val_acc:.4f}")
        
        # Plot metrics every 5 epochs
        if (epoch + 1) % 5 == 0:
            plot_metrics(train_losses, val_metrics)
            
except Exception as e:
    print(f"Error during training: {str(e)}")
    raise

## Final Evaluation

In [ ]:
# Plot final training curves
plot_metrics(train_losses, val_metrics)

# Print best validation accuracy
print(f'Best validation accuracy: {best_val_acc:.4f}')